In [12]:
from data_collection import *
import geemap

/Users/placiermoise/Documents/dsw_proj/remote sensing crop classification/.venv/lib/python3.12/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
# Load the shapefile containing the parcels
gdf = gpd.read_file("/Users/placiermoise/Documents/dsw_proj/Parcelles/RPG_2023/PARCELLES_GRAPHIQUES.gpkg")
gdf  = gdf.to_crs(epsg=4326)
Label_culture = pd.read_csv("../data/REF_CULTURES_GROUPES.csv", sep=";")
Label_culture["Label"] = pd.to_numeric(Label_culture["Label"], errors="coerce").astype("Int64")
Label_culture["Label"] = Label_culture["Label"].fillna(0).astype(int)
gdf = gdf.merge(Label_culture[["Label","CODE_CULTURE"]], how="left", left_on="CODE_CULTU", right_on="CODE_CULTURE") #TODO check if this is right (CODE_CULTU)
gdf = gdf.rename(columns={"Label": "LABEL"})

In [7]:
ee.Authenticate(scopes=['https://www.googleapis.com/auth/earthengine',
                        'https://www.googleapis.com/auth/drive'])
ee.Initialize()

In [14]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
coordinates = []
for feature in Map.draw_features:
        coordinate = feature.geometry().getInfo()
        coordinates.append(coordinate['coordinates'])
coordinates  

In [8]:
coords_list = [[0.472412, 44.504341],
 [0.692139, 46.149394],
 [5.218506, 45.413876],
 [1.450195, 48.166085]]


List_Geometry_data_collect = create_gee_geometries(coords_list, side_km=10)

In [ ]:

start_date = '2023-01-01'
end_date = '2023-12-31'

#ranges = generate_half_month_ranges(start_date, end_date)
ranges = generate_date_ranges(start_date, end_date, interval_days=30)

bands = ['B2', 'B3', 'B4', 'B8']


zone_number = 0

for geometry in List_Geometry_data_collect : 

    zone_number += 1

    # Create a list of median images for each date range
    median_images = [get_median_image(start, end, geometry) for start, end in ranges]

    # Create a collection from the images
    image_collection = ee.ImageCollection(median_images)

    folderName = f'GEE_export_zone{zone_number}'

    # export to TIFF format
    export_folder = str(folderName)
    export_maxPixels = 1e10
    
    export_tiff(image_collection,geometry,export_folder,export_maxPixels )

######### ajout manuel des files gg drive vers ../data/tif

In [ ]:
rasterize_labels_per_zone(base_dir="../data/Tif", gdf=gdf, label_column="LABEL")